(time-series)=
# Time Series

In this chapter, we'll look at time series. If you haven't yet looked at the two sections on **pandas**, the [Data Quickstart](data-quickstart) and [Working with Data](working-with-data) chapters, it might be worth taking a quick spin through them first. You may also find it useful to be familiar with a few of the concepts from the previous [chapter on time](time-intro).

While we'll cover the basics here, the full set of time series functionality of **pandas** can be [found here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html).

This chapter has benefitted from the [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) by Jake VanderPlas, Tom Augspurger's [Effective Pandas](https://github.com/TomAugspurger/effective-pandas), and [Applied Time Series for Fisheries and Environmental Sciences](https://nwfsc-timeseries.github.io/atsa-labs/).

Let's imports a few of the packages we'll need first.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
from rich import inspect
import matplotlib.pyplot as plt

# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
# Set max rows displayed for readability
pd.set_option("display.max_rows", 6)

## Introducing Time Series with **pandas**

[**pandas**](https://pandas.pydata.org/) is the workhorse of time series analysis in Python. The basic object is a *timestamp*. The `pd.to_datetime` function creates timestamps from strings that could reasonably represent datetimes. Let's see an example of using `pd.to_datetime` to create a timestamp and then inspect all of the methods and attributes of the created timestamp using **rich**'s `inspect` function.

In [ ]:
date = pd.to_datetime("16th of February, 2020")
inspect(date)

This is of type `Timestamp` and you can see that it has many of the same properties as the built-in Python `datetime.datetime` class from the previous chapter. As with that, the default setting for `tz` (timezone) and `tzinfo` is `None`. There are some extra properties, though, such as `freq` for frequency, which will be very useful when it comes to manipulating time *series* as opposed to just one or two datetimes.

## Creating Time Series

There are two main scenarios in which you might be creating time series using **pandas**: i) creating one from scratch or ii) reading in data from a file. Let's look at a few ways to do i) first. 

You can create a time series with **pandas** by taking a date as created above and extending it using **pandas** timedelta function:

In [ ]:
date + pd.to_timedelta(np.arange(12), "D")

This has created a datetime index of type `datetime65[ns]` (remember, an index is a special type of **pandas** column), where "ns" stands for nano-second resolution.

Another method is to create a range of dates (pass a frequency using the `freq=` keyword argument):

In [ ]:
pd.date_range(start="2018/1/1", end="2018/1/8")

Another way to create ranges is to specify the number of periods and the frequency:

In [ ]:
pd.date_range("2018-01-01", periods=3, freq="H")

Following the discussion of the previous chapter on timezones, you can also localise timezones directly in **pandas** dataframes:


In [ ]:
dti = pd.date_range("2018-01-01", periods=3, freq="H").tz_localize("UTC")
dti.tz_convert("US/Pacific")

Now let's see how to turn data that has been read in with a non-datetime type into a vector of datetimes. This happens *all the time* in practice. We'll read in some data on job vacancies for information and communication jobs, ONS code UNEM-JP9P, and then try to wrangle the given "date" column into a **pandas** datetime column.

In [ ]:
import requests

url = "https://api.ons.gov.uk/timeseries/JP9P/dataset/UNEM/data"

# Get the data from the ONS API:
df = pd.DataFrame(pd.json_normalize(requests.get(url).json()["months"]))
df["value"] = pd.to_numeric(df["value"])
df = df[["date", "value"]]
df = df.rename(columns={"value": "Vacancies (ICT), thousands"})
df.head()

We have the data in. Let's look at the column types that arrived.

In [ ]:
df.info()

This is the default 'object' type, but we want the date column to have `datetime64[ns]`, which is a datetime type. Again, we use `pd.to_datetime`:

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df["date"].head()

In this case, the conversion from the format of data that was put in of "2001 MAY" to datetime worked out-of-the-box. `pd.to_datetime` will always take an educated guess as to the format, but it won't always work out.

What happens if we have a more tricky-to-read-in datetime column? This frequently occurs in practice so it's well worth exploring an example. Let's create some random data with dates in an unusual format with month first, then year, then day, eg "1, '19, 29" and so on.

In [ ]:
small_df = pd.DataFrame({"date": ["1, '19, 22", "1, '19, 23"], "values": ["1", "2"]})
small_df["date"]

Now, if we were to run this via `pd.to_datetime` with no further input, it would misinterpret, for example, the first date as `2022-01-19`. So we must pass a bit more info to `pd.to_datetime` to help it out. We can pass a `format=` keyword argument with the format that the datetime takes. Here, we'll use `%m` for month in number format, `%y` for year in 2-digit format, and `%d` for 2-digit day. We can also add in the other characters such as `'` and `,`. You can find a list of datetime format identifiers in the previous chapter or over at [https://strftime.org/](https://strftime.org/).

In [ ]:
pd.to_datetime(small_df["date"], format="%m, '%y, %d")

## Datetime Offsets

Our data, currently held in `df`, were read in as if they were from the *start* of the month but these data refer to the month that has passed and so should be for the *end* of the month. Fortunately, we can change this using a time offset.

In [ ]:
df["date"] = df["date"] + pd.offsets.MonthEnd()
df.head()

While we used the `MonthEnd` offset here, there are many different offsets available. You can find a [full table of date offsets here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).

## Creating a datetime index and setting the frequency

For the subsequent parts, we'll set the datetime column to be the index of the dataframe. *This is the standard setup you will likely want to use when dealing with time series.*

In [ ]:
df = df.set_index("date")
df.head()

Now, if we look at the first few entries of the index of dataframe (a datetime index) using `head` as above, we'll see that the `freq=` parameter is set as `None`.

In [ ]:
df.index[:5]

This can be set for the whole dataframe using the `asfreq` function:

In [ ]:
df = df.asfreq("M")
df.index[:5]

Although most of the time it doesn't matter about the fact that `freq=None`, some aggregation operations need to know the frequency of the time series in order to work and it's good practice to set it if your data *are* regular. You can use `asfreq` to go from a higher frequency to a lower frequency too: the last entry from the higher frequency that aligns with the lower frequency will be taken, for example in going from months to years, December's value would be used.

Note that trying to set the frequency when your datetime index doesn't match up to a particular frequency will cause errors or problems.

A few useful frequencies to know about are in the table below; all of these can be used with `pd.to_datetime` too.

| Code  | Represents                                                          |
|-------|---------------------------------------------------------------------|
| D     | Calendar day                                                        |
| W     | Weekly                                                              |
| M     | Month end                                                           |
| Q     | Quarter end                                                         |
| A     | Year end                                                            |
| H     | Hours                                                               |
| T     | Minutes                                                             |
| S     | Seconds                                                             |
| B     | Business day                                                        |
| BM    | Business month end                                                  |
| BQ    | Business quarter end                                                |
| BA    | Business year end                                                   |
| BH    | Business hours                                                      |
| MS    | Month start                                                         |
| QS    | Quarter start                                                       |
| W-SUN | Weeks beginning with Sunday (similar for other days)                |
| 2M    | Every 2 months (works with other combinations of numbers and codes) |

## Making Quick Time Series Plots

Having managed to put your time series into a dataframe, perhaps converting a column of type string into a colume of type datetime in the process, you often just want to see the thing! We can achieve this using the `plot` command, as long as we have a datetime index.


In [ ]:
df.plot();

## Resampling, Rolling, and Shifting

Now our data have a *datetime index*, some common time series operations are made very easy for us.

### Resampling

Quite frequently, there is a situation in which one would like to change the frequency of a given time series. A time index-based dataframe makes this easy via the `resample` function. `resample` must be told *how* you'd like to resample the data, for example via the mean or median. Here's an example resampling the monthly data to annual and taking the mean:

In [ ]:
df.resample("A").mean()

As resample is just a special type of aggregation, it can work with all of the usual functions that aggregations do, including in-built functions or user-defined functions.

In [ ]:
df.resample("5A").agg(["mean", "std"]).head()

Resampling can go up in frequency (up-sampling) as well as down, but we no longer need to choose an aggregation function, we must now choose how we'd like to fill in the gaps for the frequencies we didn't have in the original data. In the example below, they are just left as NaNs.

In [ ]:
df.resample("D").asfreq()

Options to fill in missing time series data include using `bfill` or `ffill` to fill in the blanks based on the next or last available value, respectively, or `interpolate` (note how only the first 3 NaNs are replaced using the `limit` keyword argument):

In [ ]:
df.resample("D").interpolate(method="linear", limit_direction="forward", limit=3)[:6]

We can see the differences between the filling methods more clearly in this stock market data, following a chart by Jake Vanderplas.

In [ ]:
# Get stock market data
import pandas_datareader as web

xf = web.DataReader("AAPL", "stooq", start="2017-01-01", end="2019-06-01")
xf = xf.sort_index()

In [ ]:
fig, ax = plt.subplots()
data = xf.iloc[:10, 3]
cycle = ax._get_lines.prop_cycler

data.asfreq("D").plot(ax=ax, marker="o", linestyle="None", zorder=3)
data.asfreq("D", method="bfill").plot(
    ax=ax, style="-.o", lw=1, color=next(cycle)["color"]
)
data.asfreq("D", method="ffill").plot(
    ax=ax, style="--o", lw=1, color=next(cycle)["color"]
)
ax.set_ylabel("Close ($)")
ax.legend(["original", "back-fill", "forward-fill"]);

### Rolling Window Functions

The `rolling` and `ewm` methods are both rolling window functions. The first includes functions of the sequence

$$
y_t = f(\{x_{t-i} \}_{i=0}^{i=R-1})
$$

where $R$ is the number of periods to use for the rolling window. For example, if the function is the mean, then $f$ takes the form $\frac{1}{R}\displaystyle\sum_{i=0}^{i=R-1} x_{t-i}$.

The example below is a 2-period rolling mean:

In [ ]:
df.rolling(2).mean()

The `ewm` includes the class of functions where data point $x_{t-i}$ has a weight $w_i = (1-\alpha)^i$. As $0 < \alpha < 1$, points further back in time are given less weight. For example, an exponentially moving average is given by

$$
y_t = \frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + ...
+ (1 - \alpha)^t x_{0}}{1 + (1 - \alpha) + (1 - \alpha)^2 + ...
+ (1 - \alpha)^t}
$$

The example below shows the code for the exponentially weighted moving average:

In [ ]:
df.ewm(alpha=0.2).mean()

Let's see these methods together on the stock market data.

In [ ]:
fig, ax = plt.subplots()
roll_num = 28
alpha = 0.03
xf["Close"].plot(label="Raw", alpha=0.5)
xf["Close"].expanding().mean().plot(label="Expanding Average", style=":")
xf["Close"].ewm(alpha=alpha).mean().plot(
    label=f"EWMA ($\\alpha=${alpha:.2f})", style="--"
)
xf["Close"].rolling(roll_num).mean().plot(label=f"{roll_num}D MA", style="-.")
ax.legend()
ax.set_ylabel("Close ($)");

For more tools to analyse stocks, see the [**Pandas TA**](https://twopirllc.github.io/pandas-ta/) package.

We can also use `rolling` as an intermediate step in creating more than one type of aggregation:

In [ ]:
roll = xf["Close"].rolling(50, center=True)

fig, ax = plt.subplots()
m = roll.agg(["mean", "std"])
m["mean"].plot(ax=ax)
ax.fill_between(m.index, m["mean"] - m["std"], m["mean"] + m["std"], alpha=0.2)
ax.set_ylabel("Close ($)");

### Shifting

Shifting can move series around in time; it's what we need to create leads and lags of time series. Let's create a lead and a lag in the data. Remember that a lead is going to shift the pattern in the data to the left (ie earlier in time), while the lag is going to shift patterns later in time (ie to the right).

In [ ]:
lead = 12
lag = 3
orig_series_name = df.columns[0]
df[f"lead ({lead} months)"] = df[orig_series_name].shift(-lead)
df[f"lag ({lag} months)"] = df[orig_series_name].shift(lag)
df.head()

In [ ]:
df.iloc[100:300, :].plot();

## Correlations

In this section, we'll look at some different ways of comparing time series using simple metrics.

### Autocorrelation

First, what about comparing a time series to itself? The autocorrelation of a time series tells you how persistent it is. The econometrics package [**statsmodels**]() has some tools for this, most notably `statsmodels.tsa.stattools.acf`. Sometimes what you want is just a visual cue though, in which case the code below produces a nice chart.

In [ ]:
fig = tsaplots.plot_acf(df["Vacancies (ICT), thousands"], lags=24)
plt.show()

Rememebr that the autocorrelation can tell you about likely terms to include in a moving average (MA) model.

We can also look at the *partial autocorrelation*, which, at a given lag, is the correlation that results after removing the effect of any correlations due to the terms at shorter lags. It can be computer via the `statsmodels.tsa.stattools.pacf` function, but here's a chart:

In [ ]:
fig = tsaplots.plot_pacf(df["Vacancies (ICT), thousands"], lags=24)
plt.show()

Remember that the partial autocorrelation function can tell you about likely terms to include in an autoregressive (AR) model.

By looking at both the ACF and PACF, you can get an idea of what terms would be good to include in *both* an MA and AR model which, combined, make an ARMA model. It's also worth noting that, if the ACF and PACF do not seem to tail off, the series is likely to be non-stationary and differencing will be needed if you are going to model it with an ARMA approach.

### Correlations with Other Series

To compute correlations with other time series, a popular choice is the cross-correlation function (CCF). Confusingly, there are different definitions of this object in statistics and signal processing. In statistics, it's given by

$$
r_k = \frac{g_k}{\sqrt{\sigma_x\sigma_y}};
$$

where

$$
g_k = \frac{1}{n}\sum_{t=1}^{n-k} \left(y_t-\bar{y}\right) \left(x_{t+k}-\bar{x}\right),
$$

and $k$ is set by a rule of thumb, usually something like $10 \log_{10}\left(\min(|x|, |y|)/2\right)$. $g_k$ is the 'sliding' dot product of $y$, the response, with $x$, the predictor, up to the $k$ th lag.

The default cross-correlation function in **statsmodels** is different. It only looks forward, and it computes *all* possible lags. You can call it via `sm.ccf` if you have already run `import statsmodels.api as sm`.

We'll use the standard stats treatment here, but we'll need to define a function to do the computation we need. Then we'll apply the CCF to a classic example: the response of the number of Lynx trapped (in Canada) to sunspot activity.

First, defining the function. We'll make use of the signal processing `ss.correlate` function but modify it so that adopts the definition of CCF used in statistics:

In [ ]:
import scipy.signal as ss


def ccf_stats(x, y, lag_max=None):
    if not lag_max:
        lag_max = int(10 * np.log10(min(len(x), len(y)) / 2))
    result = ss.correlate(x - np.mean(x), y - np.mean(y), method="direct") / (
        np.std(y) * np.std(x) * len(y)
    )
    length = (len(result) - 1) // 2
    lo = length - lag_max
    hi = length + (lag_max + 1)
    return result[lo:hi]

Now let's grab the two different datasets and merge them, taking a quick look while we're at it:

In [ ]:
lynx_df = pd.read_csv(
    "https://vincentarelbundock.github.io/Rdatasets/csv/datasets/lynx.csv", index_col=0
)
spot_df = pd.read_csv(
    "https://vincentarelbundock.github.io/Rdatasets/csv/datasets/sunspot.year.csv",
    index_col=0,
)
df = pd.merge(lynx_df, spot_df, on="time", how="inner", suffixes=("_lynx", "_sunspots"))
df = df.set_index("time")
df.head()

In [ ]:
df.plot(subplots=True, ylim=(0, None));

Let's use our defined function on the data. We'll add in a rough confidence interval of $\pm \frac{2}{\sqrt{n}}$.

In [ ]:
fig, ax = plt.subplots()
ccf_res_stats = ccf_stats(df["value_sunspots"], np.log(df["value_lynx"]))
conf_int = 2.0 / np.sqrt(len(df))
ax.stem(
    range(-int(len(ccf_res_stats) / 2), int(len(ccf_res_stats) / 2) + 1),
    ccf_res_stats,
    basefmt=" ",
    markerfmt=" ",
)
ax.axhline(-conf_int, ls="--", lw=0.9, color="k")
ax.axhline(conf_int, ls="--", lw=0.9, color="k")
ax.axhline(0, color="k", lw=1)
ax.set_ylabel("Cross-correlation")
ax.set_xlabel("Lag")
plt.show()

The significant correlations at lags of -3 to -5 seem to suggest that lynx numbers fall in the 3 to 5 years after high sunspot activity.

## Transformations

There are many reasons why you might want to perform transformations on series, including to normalise data and to adjust for changes (such as inflation). In this section, we'll look at some of the most common transforms for time series.

### Mathematical Transformations

The usual transformations that we know and love from other series of numerical data can equally be applied to time series, for example the log and power transforms. Let's see some of these in action using data on US GDP.

In [ ]:
import pandas_datareader.data as web
from datetime import datetime

ts_start_date = pd.to_datetime("2009-01-01")

df = web.DataReader("NA000334Q", "fred", start=ts_start_date, end=datetime.now())
df.columns = ["gdp"]
df.head()

Now let's create new columns for each of log, square, and square root transforms. All of these mathematical transformations can be found in **numpy**.

In [ ]:
df["log"] = np.log(df["gdp"])
df["square"] = np.power(df["gdp"], 2)
df["sqrt"] = np.sqrt(df["gdp"])
df.head()

A very common transformation is to change to growth space. In time series maths, this is $\frac{y_t}{y_{t-1}}$ for period-on-period growth or, say, $\frac{y_t}{y_{t-4}}$ for growth relative to 4 periods ago.

Let's add in quarter-on-quarter-a-year-ago and quarter-on-quarter growth rates to our growing list of transforms:

In [ ]:
df["q_on_q"] = df["gdp"].pct_change(1) * 100
df["q_on_q_yr_ago"] = df["gdp"].pct_change(4) * 100
df.tail()

Another classic transform you might need is to de-mean your data. We are transforming such that

$$
y_t' = y_t - \langle y_t \rangle
$$

where the angular brackets denote a mean. Here's an example of that:

In [ ]:
df["demean"] = df["gdp"] - df["gdp"].mean()
df[["gdp", "demean"]].tail()

Be wary though! The mean is a function that, by default, uses all time periods, so you need to be careful if you're working on a real-time data problem such as forecasting. In this case you only want to use the mean *up to that point in time* to demean a series. Mathematically, this is

$$
y_t' = y_t - \frac{1}{t+1}\displaystyle{\sum_{\tau=0}^{\tau=t} y_\tau}
$$

To do this, we can make use of the **pandas** *expanding* functionality we saw earlier. Here's an example of real-time demeaning:

In [ ]:
df["real_time_demean"] = df["gdp"] - df["gdp"].expanding().mean()
df[["gdp", "real_time_demean"]].head()

Note how the second real-time demeaned entry is the second entry for GDP minus the average of the second and first entries of GDP.

Let's move on to another classic transformation: differences. These are given by

$$
\Delta y_t = y_t - y_{t-1}
$$

Note that, in this case, the first entry in the differences won't be defined (ie appears as `NaN`) because there is no $\text{gdp}_{t=-1}$. Use `diff(n)` to get $y_t - y_{t-n}$.

In [ ]:
df["difference"] = df["gdp"].diff()
df[["gdp", "difference"]].head()

### The Box-Cox Transform

(ipywidget here?)